Download dataset

made with Python 3.10.4

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

Couldn't download the data with code, because it timed out. So we load the downloaded data in with the image_dataset_from_directory function

creating a new file (data.csv), with better format to read in with a generator, values are separated by coma and each line contains the file name, 1 or 0 for each attribute, 

In [7]:
#only need to run once, than use data.csv file to load labels
Y = []
file_path = 'data.csv'

with open("list_attr_celeba.txt") as attr:
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        count = 0
        size = next(attr)
        labels = next(attr).strip()
        firstRow = ["Filenames"] + labels.split()
        writer.writerow(firstRow)
        for line in attr:
            name = line[:11]
            line = line[11:]
            numbers = line.split()
            for (i, number) in enumerate(numbers):
                if number == "-1":
                    numbers[i] = 0
                elif number == "1":
                    numbers[i] = 1
            data = [name] + numbers
            writer.writerow(data)
            

In [ ]:
with open("list_attr_celeba.txt") as attr:
        count = 0
        size = next(attr)
        labels = next(attr).strip()
labels = labels.split()

load data with generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

right now, only use part of the data, so it's easier to handle

In [ ]:
df=pd.read_csv("./img_align_celeba/data.csv")
columns=labels
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
dataframe=df[:1800],
directory="./img_align_celeba/img_align_celeba",
x_col="Filenames",
validate_filenames=False,
y_col=columns,
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(178,218))

valid_generator=test_datagen.flow_from_dataframe(
dataframe=df[1800:1900],
directory="./img_align_celeba/img_align_celeba",
x_col="Filenames",
validate_filenames=False,
y_col=columns,
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(178,218))

test_generator=test_datagen.flow_from_dataframe(
dataframe=df[1900:2000],
directory="./img_align_celeba/img_align_celeba",
x_col="Filenames",
validate_filenames=False,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(178,218))

Found 1800 non-validated image filenames.
Found 100 non-validated image filenames.
Found 100 non-validated image filenames.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(178, 218, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(40, activation='sigmoid')  # Assuming there are 40 facial attributes to classify
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 176, 216, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 88, 108, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 86, 106, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 43, 53, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 51, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 20, 25, 128)       0

In [ ]:
#todo change from model.fit_generator to model.fit because it is deprecated
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

C:\Users\Zear\AppData\Local\Temp\ipykernel_2248\3771039389.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
56/56 [==============================] - 72s 1s/step - loss: 0.4411 - accuracy: 0.0090 - val_loss: 0.4006 - val_accuracy: 0.0000e+00
Epoch 2/10
56/56 [==============================] - 61s 1s/step - loss: 0.3781 - accuracy: 0.0045 - val_loss: 0.3510 - val_accuracy: 0.0104
Epoch 3/10
56/56 [==============================] - 60s 1s/step - loss: 0.3201 - accuracy: 0.0130 - val_loss: 0.3286 - val_accuracy: 0.0104
Epoch 4/10
56/56 [==============================] - 65s 1s/step - loss: 0.2737 - accuracy: 0.0147 - val_loss: 0.3199 - val_accuracy: 0.0104
Epoch 5/10
56/56 [==============================] - 59s 1s/step - loss: 0.2337 - accuracy: 0.0164 - val_loss: 0.3514 - val_accuracy: 0.0104
Epoch 6/10
56/56 [==============================] - 48s 853ms/step - loss: 0.1890 - accuracy: 0.0192 - val_loss: 0.3731 - val_accuracy: 0.0208
Epoch 7/10
56/56 [==============================] - 47s 845ms/step - loss: 0.1417 - accuracy: 0.0175 - val_loss: 0.4342 - val_accuracy: 0.0208
Epoch 8/10